# Lecture du Fichier de coordonnées de l'ISS

## [TODO] Introduction

https://spotthestation.nasa.gov/trajectory_data.cfm

After the header, ISS state vectors in the Mean of J2000 (J2K) reference frame are listed at four-minute intervals spanning a total length of 15 days. During reboosts (translation maneuvers), the state vectors are reported in two-second intervals. Each state vector lists the time in UTC; position X, Y, and Z in km; and velocity X, Y, and Z in km/s.

Orbit Ephemeris Message (OEM)

    https://nasa-public-data.s3.amazonaws.com/iss-coords/current/ISS_OEM/ISS.OEM_J2K_EPH.txt

In [1]:
import glob

# Récupération d'un fichier de données présent, peu importe lequel.
filename = glob.glob("ISS.OEM_J2K_EPH_*.txt")[0]

# Affichage des 50 premières lignes du fichier
with open(filename) as file:
    file_lines = file.readlines()
    for line in file_lines[:50]:
        print(line, end='')

CCSDS_OEM_VERS = 2.0
CREATION_DATE  = 2023-04-17T23:02:52.225
ORIGINATOR     = NASA/JSC/FOD/TOPO

META_START
OBJECT_NAME          = ISS
OBJECT_ID            = 1998-067-A
CENTER_NAME          = Earth
REF_FRAME            = EME2000
TIME_SYSTEM          = UTC
START_TIME           = 2023-04-17T12:00:00.000
USEABLE_START_TIME   = 2023-04-17T12:00:00.000
USEABLE_STOP_TIME    = 2023-05-02T12:00:00.000
STOP_TIME            = 2023-05-02T12:00:00.000
META_STOP

COMMENT Source: This file was produced by the TOPO office within FOD at JSC.
COMMENT Units are in kg and m^2
COMMENT MASS=454160.00
COMMENT DRAG_AREA=1450.15
COMMENT DRAG_COEFF=2.35
COMMENT SOLAR_RAD_AREA=0.00
COMMENT SOLAR_RAD_COEFF=0.00
COMMENT Orbits start at the ascending node epoch
COMMENT ISS first asc. node: EPOCH = 2023-04-17T12:31:51.862 $ ORBIT = 3237 $ LAN(DEG) = -124.17695
COMMENT ISS last asc. node : EPOCH = 2023-05-02T11:29:00.999 $ ORBIT = 3469 $ LAN(DEG) = 162.68204
COMMENT Begin sequence of events
COMMENT TRAJECTORY EVENT

[TODO] Explications du contenu du fichier

## Différenciation des phases de poussée

In [2]:
import pandas as pd

# Sélection des lignes de commentaires
comments = [line for line in file_lines if line.startswith("COMMENT")]
# Stockage de l'index de la dernière ligne de commentaire en prévision de l'accès aux données
last_comment_index = file_lines.index(comments[-1])
# Stockage des données dans un DataFrame, à partir des lignes suivant le dernier commentaire
df = pd.read_csv(filename, skiprows=last_comment_index+1,
                 sep=" ", names=['datetime', 'x', 'y', 'z', 'vx', 'vy', 'vz'],
                 parse_dates=[0], infer_datetime_format=True)

df.datetime.diff().groupby(by=df.datetime.diff().rename('timedelta')).count().to_frame('count')

,count
timedelta,
0 days 00:00:00.748000,1
0 days 00:00:02,367
0 days 00:02:00,2
0 days 00:03:45.252000,1
0 days 00:04:00,5395


In [3]:
# Ajout d'une colonne spécifiant le nombre de minisecondes depuis la dernière position
df['delta'] = (df.diff().datetime.dt.total_seconds() * 1000).fillna(-1).astype(int)

In [4]:
# Affichage des deux enregistrements précédant et suivant les delta anormaux
for i in df[1:].query("delta != 2000 and delta != 240000").index:
    display(df[i-2:i+3].style.apply(lambda r:(r.size-1)*['']+['background-color: pink'
                                                              if i==r.name else ''], axis = 1))

,datetime,x,y,z,vx,vy,vz,delta
825,2023-04-19 19:00:00,3481.701537,-2902.931539,5056.924941,2.991968,6.808008,1.839873,240000
826,2023-04-19 19:04:00,4064.545510,-1183.369294,5308.851765,1.835403,7.434270,0.246634,240000
827,2023-04-19 19:06:00,4247.024857,-283.172548,5289.764664,1.201263,7.546147,-0.564294,120000
828,2023-04-19 19:10:00,4377.423988,1516.193272,4963.130522,-0.121389,7.357016,-2.141183,240000
829,2023-04-19 19:14:00,4189.378416,3205.242657,4274.421382,-1.436284,6.632287,-3.563114,240000


,datetime,x,y,z,vx,vy,vz,delta
1429,2023-04-21 11:14:00,3067.440617,-3307.049616,5077.117930,3.903514,6.349238,1.769591,240000
1430,2023-04-21 11:18:00,3881.466064,-1681.574786,5311.661961,2.838619,7.113781,0.172940,240000
1431,2023-04-21 11:20:00,4185.638999,-815.171508,5283.735364,2.223180,7.304273,-0.637703,120000
1432,2023-04-21 11:24:00,4560.559401,946.191426,4939.924296,0.881929,7.284148,-2.210032,240000
1433,2023-04-21 11:28:00,4603.728001,2638.706114,4235.748967,-0.524611,6.733888,-3.622366,240000


,datetime,x,y,z,vx,vy,vz,delta
4871,2023-04-30 00:28:12,-5444.680221,790.752582,-3987.934808,-3.698014,-5.402672,3.974361,2000
4872,2023-04-30 00:28:14,-5452.062425,779.945239,-3979.975939,-3.684186,-5.404667,3.984504,2000
4873,2023-04-30 00:28:14.748000,-5454.816260,775.902271,-3976.994113,-3.679010,-5.405406,3.988292,748
4874,2023-04-30 00:32:14.748000,-6128.827786,-533.833358,-2886.525802,-1.903195,-5.442467,5.043501,240000
4875,2023-04-30 00:36:14.748000,-6357.118026,-1804.731049,-1585.549447,0.012641,-5.083620,5.731653,240000


,datetime,x,y,z,vx,vy,vz,delta
5764,2023-05-02 11:52:14.748000,1091.158230,-4077.374683,5323.151466,7.405811,1.947562,-0.025127,240000
5765,2023-05-02 11:56:14.748000,2807.408526,-3467.684075,5123.496433,6.809133,3.102267,-1.628538,240000
5766,2023-05-02 12:00:00,4234.964203,-2665.415866,4596.207153,5.797933,3.982879,-3.028158,225252


Les écarts non-standards débutent ou terminent les poussées, ou sont sporatiques dans le jeu de données.
On retiendra donc uniquement les intervales de 2 secondes pour représenter les poussées.

In [5]:
# poussée

In [6]:
import numpy as np

# Les enregistrements datant de deux secondes après leur précédent sont marqués en tant que poussée
df['on_thrust'] = np.isclose(df.diff().datetime.dt.total_seconds(), 2)

# Initialisation des valeurs d'épisode de poussée
current_state = df.iloc[0]['on_thrust']
current_tid = 0

def thrust_episode(on_thrust):
    """Fonction pour le parours de proche en proche des états de poussée pour la définition d'épisodes."""
    global current_state, current_tid
    # Si l'état actuel est différent du dernier état enregistré
    if on_thrust != current_state:
        # Changement d'état et d'identifiant d'épisode.
        current_state = on_thrust
        current_tid += 1
    # Renvoi de l'indentifiant actuel
    return current_tid

# Un épsiode est une séquence contiguë d'états de poussée
df['thrust_episode'] = df['on_thrust'].apply(thrust_episode)

# Décompte des épisodes identifiés
df.groupby('thrust_episode').count()[['on_thrust']]

,on_thrust
thrust_episode,
0,4506
1,367
2,894


## Classe pour la bibliothèque

Pour permettre l'utilisation simple des données de ce ficiher, nous définissons une classe `ISS_Position` pour notre bibliothèque. Une instance de cette classe permet d'accéder directement aux métadonnées, commentaires et données contenus dans le fichier par des méthodes respectivement nommées `get_metadata()`, `get_comments()` et `get_data()`.

In [23]:
import os
import re
import requests


class ISS_Position:
    """Classe d'extraction des données de position de l'ISS."""
    def __init__(self, filename=None, force_download=False):
        """Réalise une extraction de données d'un fichier source de coordonnées de l'ISS.
        
        La lecture s'effectue depuis un fichier local si spécifié. Dans le cas contraire, le fichier est téléchargé
        depuis `https://nasa-public-data.s3.amazonaws.com/iss-coords/current/ISS_OEM/ISS.OEM_J2K_EPH.txt` si il
        n'existe pas déjà un téléchargement local de ce fichier.
        
        Paramètres:
        - filename: Spécifie le fichier local à utiliser comme source de données.
        - force_download: Force le téléchargement du fichier même s'il existe déjà en local.
        """
        if not filename:
            # En l'absence de fichié spécifié, génère le nom de fichier local du jour
            filename = f"ISS.OEM_J2K_EPH_{pd.Timestamp.today().strftime('%Y%m%d')}.txt"
            # Télécharge le fichier du jour s'il est absent ou si le téléchargement est forcé
            if force_download or not os.path.exists(filename):
                self.__download(filename)
        # Parse le contenu du fichier
        self.__parse_source(filename)
    
    def __download(self, filename):
        """Télécharge le fichier de coordonnées de l'ISS depuis le serveur public de la NASA."""
        # Permalien du fichier
        url = "https://nasa-public-data.s3.amazonaws.com/iss-coords/current/ISS_OEM/ISS.OEM_J2K_EPH.txt"
        # Téléchargement en mémoire du fichier
        myfile = requests.get(url)
        # Ecriture en fichier local
        with open(filename, 'wb') as file:
            file.write(myfile.content)
        
    def __parse_source(self, filename):
        """Analyse syntaxique du fichier."""
        with open(filename) as file:
            source_lines = file.readlines()
        # Extraction des métadonnées
        self.__parse_source_meta(source_lines)
        # Extraction des commentaires
        self.__parse_source_comments(source_lines)
        # Extraction des données
        self.__parse_data(filename)
    
    def __parse_source_meta(self, source_lines):
        """Extraction des métadonnées du fichier."""
        # Détection des lignes de métadonnées
        meta = slice(source_lines.index('META_START\n')+1, source_lines.index('META_STOP\n'))
        # Enregistrement des métadonnées dans un dictionnaire
        self.meta = {key.strip(): pd.to_datetime(value.strip()) if key.strip().endswith('_TIME') else value.strip()
                     for line in source_lines[meta]
                     for (key, value) in [line.split('=', 1)]
                    }
    
    def __parse_source_comments(self, source_lines):
        """Extraction des commentaires du fichier."""
        # Sélection des lignes de commentaires
        comments = [line for line in source_lines if line.startswith("COMMENT")]
        # Stockage de l'index de la dernière ligne de commentaire en prévision de l'accès aux données
        self.__last_comment_index = source_lines.index(comments[-1])
        # Enregistrement des commentaires dans une liste
        self.comments = [comment[7:].strip() for comment in comments]
        # Recherche par motif des metadonnées inclues dans les commentaires
        md_comments = re.findall(r"([A-Z_]+)=([0-9\.]+)", '\n'.join(self.comments))
        # Ajout de ces métadonnées aux dictonnaire des métadonnées
        self.meta.update({match[0]:float(match[1]) for match in md_comments})
    
    def __parse_data(self, filename):
        """Extraction des données du fichier."""
        # Enregistrement des données dans un DataFrame, à partir des lignes suivant le dernier commentaire
        self.data = pd.read_csv(filename, skiprows=self.__last_comment_index+1,
                                sep=" ", names=['datetime', 'x', 'y', 'z', 'vx', 'vy', 'vz'],
                                parse_dates=[0], infer_datetime_format=True)
        # Ajoute les annotations de poussée
        self.__add_thrust_metadata()
        
    def __add_thrust_metadata(self):
        """Ajout des annotation de poussé et épisodes de poussée."""
        # Les enregistrements datant de deux secondes après leur précédent sont marqués en tant que poussée
        self.data['on_thrust'] = np.isclose(self.data.diff().datetime.dt.total_seconds(), 2)
        # Initialisation des valeurs d'épisode de poussée
        self.__current_state = self.data.iloc[0]['on_thrust']
        self.__current_tid = 0
        # Un épsiode est une séquence contiguë d'état de poussée
        self.data['thrust_episode'] = self.data['on_thrust'].apply(self.__thrust_episode)
        # Suppression des marqueurs inutiles
        del self.__current_state, self.__current_tid

    def __thrust_episode(self, on_thrust):
        """Méthode pour le parours de proche en proche des états de poussée pour la définition d'épisodes."""
        # Si l'état actuel est différent du dernier état enregistré
        if on_thrust != self.__current_state:
            # Changement d'état et d'identifiant d'épisode.
            self.__current_state = on_thrust
            self.__current_tid += 1
        # Renvoi de l'indentifiant actuel
        return self.__current_tid

    def get_metadata(self, key=None):
        """Accès aux métadonnées.
        
        Si une clé est spécifiée et qu'elle existe dans les métadonnées, retourne sa valeur.
        Si aucune clé n'est fournie, retourne l'ensemble des métadonnées
        en utilisant un DataFrame (pour son rendu élégant à l'affichage).
        """
        if key:
            return self.meta.get(key)
        else:
            return pd.DataFrame(self.meta.values(), self.meta.keys(), ['Metadata'])
    
    def get_comments(self):
        """Retourne les commentaires en tant que bloc de texte avec retours à la ligne."""
        return '\n'.join(self.comments)
    
    def get_data(self):
        """Retourne le DataFrame des données"""
        return self.data

    def __repr__(self):
        """Représentation en string de l'objet."""
        return fr"ISS coordinates from {self.meta['START_TIME']} to {self.meta['STOP_TIME']}"
    
    def _repr_html_(self):
        """Représentation riche en HTML de l'objet."""
        start_date = self.meta['START_TIME'].strftime('%d/%m/%Y')
        stop_date = self.meta['STOP_TIME'].strftime('%d/%m/%Y') 
        return (r"<div style='border:4px #eee outset;background:#fcfcfc;padding:1px 10px 10px'>"
                fr"<h4 style='padding-left:10px'>Coordonnées de l'ISS du {start_date} au {stop_date}</h4>"
                fr"{self.__repr_thrust_episode()}</div>")

    def __repr_thrust_episode(self):
        """Représentation des épisodes de poussées en tant que DataFrame."""
        df = pd.DataFrame([self.data.groupby('thrust_episode').min().datetime.rename('Début'),
                           self.data.groupby('thrust_episode').max().datetime.rename('Fin')]).T
        df['Durée'] = df['Fin'] - df['Début']
        df.index = df.index.map(lambda x: f"Episode {x} : " + ("Poussée moteurs" if x%2
                                                               else "Mouvement libre")).rename(None)
        return df.to_html()

## Test de la classe

Pour tester notre classe, nous l'instancions et appellons ses méthodes publiques.

In [8]:
# Instanciation de la classe
iss_position = ISS_Position()
# Test de représentation
iss_position

,Début,Fin,Durée
Episode 0 : Mouvement libre,2023-04-17 12:00:00.000,2023-04-30 00:16:00,12 days 12:16:00
Episode 1 : Poussée moteurs,2023-04-30 00:16:02.000,2023-04-30 00:28:14,0 days 00:12:12
Episode 2 : Mouvement libre,2023-04-30 00:28:14.748,2023-05-02 12:00:00,2 days 11:31:45.252000


In [9]:
# Accès aux métadonnées
iss_position.get_metadata()

,Metadata
OBJECT_NAME,ISS
OBJECT_ID,1998-067-A
CENTER_NAME,Earth
REF_FRAME,EME2000
TIME_SYSTEM,UTC
START_TIME,2023-04-17 12:00:00
USEABLE_START_TIME,2023-04-17 12:00:00
USEABLE_STOP_TIME,2023-05-02 12:00:00
STOP_TIME,2023-05-02 12:00:00


In [10]:
# Accès aux commentaires
print(iss_position.get_comments())

Source: This file was produced by the TOPO office within FOD at JSC.
Units are in kg and m^2
MASS=454160.00
DRAG_AREA=1450.15
DRAG_COEFF=2.35
SOLAR_RAD_AREA=0.00
SOLAR_RAD_COEFF=0.00
Orbits start at the ascending node epoch
ISS first asc. node: EPOCH = 2023-04-17T12:31:51.862 $ ORBIT = 3237 $ LAN(DEG) = -124.17695
ISS last asc. node : EPOCH = 2023-05-02T11:29:00.999 $ ORBIT = 3469 $ LAN(DEG) = 162.68204
Begin sequence of events
TRAJECTORY EVENT SUMMARY:

|       EVENT        |       TIG        | ORB |   DV    |   HA    |   HP    |
|                    |       GMT        |     |   M/S   |   KM    |   KM    |
|                    |                  |     |  (F/S)  |  (NM)   |  (NM)   |
NG-18 Release         111:11:20:00.000             0.0     420.9     411.7
(0.0)   (227.3)   (222.3)

Reboost GMT 120       120:00:16:00.000             1.2     417.1     412.5
(3.9)   (225.2)   (222.7)

End sequence of events


In [11]:
# Accès aux données
iss_position.get_data()

,datetime,x,y,z,vx,vy,vz,on_thrust,thrust_episode
0,2023-04-17 12:00:00.000,-3466.874794,3800.561323,-4440.756071,-2.735799,-6.344510,-3.300500,False,0
1,2023-04-17 12:04:00.000,-3989.741197,2158.461676,-5061.697302,-1.595153,-7.256123,-1.842742,False,0
2,2023-04-17 12:08:00.000,-4223.385211,359.865752,-5314.665526,-0.340222,-7.640959,-0.252696,False,0
3,2023-04-17 12:12:00.000,-4151.129459,-1464.808908,-5181.590276,0.938638,-7.472240,1.354835,False,0
4,2023-04-17 12:16:00.000,-3778.283404,-3183.412713,-4672.224777,2.149599,-6.762349,2.864091,False,0
...,...,...,...,...,...,...,...,...,...
5762,2023-05-02 11:44:14.748,-2448.585981,-4386.177275,4573.789954,6.982173,-0.692860,3.071412,False,2
5763,2023-05-02 11:48:14.748,-704.281059,-4391.166943,5135.399908,7.465029,0.651364,1.580248,False,2
5764,2023-05-02 11:52:14.748,1091.158230,-4077.374683,5323.151466,7.405811,1.947562,-0.025127,False,2
5765,2023-05-02 11:56:14.748,2807.408526,-3467.684075,5123.496433,6.809133,3.102267,-1.628538,False,2
